#### **Nome:** Leonardo Rossi Dourado
#### **RA:** 800208

#**O Problema do Caixeiro Viajante - The Travelling Salesman Problem (TSP)**

## **Descrição do Problema**

  Trata-se de um problema clássico de otimização computacional, onde o objetivo é determinar o caminho mais curto que um vendedor deve percorrer para visitar um conjunto de cidades exatamente uma vez e, em seguida, retornar à cidade de origem.

## **Formulação Matemática**
  O objetivo é que a soma total seja a mínima possível. Caso o número de cidades seja N, podemos modelar a soma da seguinte forma:

#$\sum_{i=0}^{N} \sum_{j=0}^{N}  d_{ij} * x_{ij} $

Onde:



*   $d_{ij} $ é a distância entre as cidades **i** e **j**.
*   $x_{ij}$ é uma variável binária que indica se o vendedor vai diretamente de **i** a **j** *(sendo 1 para sim e 0 para não)*.

## **Restrições**



1.   Cada cidade deve ser visitada exatamente uma vez
2.   De cada cidade, deve haver exatamente uma aresta saindo e uma aresta chegando.
3.  Evitar subciclos (ciclos que não cobrem todas as cidades).





# **Algoritmo Genético para Resolução de TSP**

De início, temos a importação da biblioteca *random* somente para a futura criação de uma população inicial aleatória.

In [1]:
import random as rd

Para a criação de dados, usaremos a matrix fornecidade pelo Google (https://developers.google.com/optimization/routing/tsp?hl=pt-br) que mostra coloca a distância real entre algumas das principais cidades dos Estados Unidos.

In [2]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]
    data["num_vehicles"] = 1
    data["depot"] = 0 # Local de origem. Neste caso, New York
    return data


data = create_data_model()

A matriz de distância é uma matriz cuja entrada **i**, **j** é a distância do local **i** ao local **j** em milhas, em que os índices da matriz correspondem aos locais na seguinte ordem:

0. New York - 1. Los Angeles - 2. Chicago - 3. Minneapolis - 4. Denver - 5. Dallas
 6. Seattle - 7. Boston - 8. San Francisco - 9. St. Louis - 10. Houston - 11. Phoenix - 12. Salt Lake City

A função fitness a seguir é medição da qualidade do caminho pela distância percorrida no trajeto do vendedor.

In [3]:
def fitness(caminho):
    distancia_total = 0
    for i in range(num_cidades - 1):
        cidade_atual = caminho[i]
        cidade_prox = caminho[i + 1]
        distancia_total += data["distance_matrix"][cidade_atual][cidade_prox]
    # Adiciona o retorno a cidade de origem
    ultima_cidade = caminho[-1]
    primeira_cidade = caminho[0]
    distancia_total += data["distance_matrix"][ultima_cidade][primeira_cidade]
    return distancia_total


Para o início do nosso algoritmo genético, precisamos de uma população inicial para começarmos o processo evolutivo. A função a seguir gera caminhos aleatórios, ou seja, uma população inicial.

In [4]:
def criar_populacao_inicial(tamanho_populacao, depot):
    populacao = [rd.sample(range(1, num_cidades), num_cidades - 1) for _ in range(tamanho_populacao)]
    # Adiciona o depot no início de cada caminho
    populacao = [[depot] + caminho for caminho in populacao]
    return populacao

Por meio de um torneio, selecionaramos os pais usando a função fitness anterior como chave

In [5]:
def selecionar_pais(populacao, tamanho_torneio):
    torneio = rd.sample(populacao, tamanho_torneio)
    vencedor = min(torneio, key=fitness)
    return vencedor

Um elemento fundamental do nosso código é o cromossomo e como eles são representados. Aqui, eles serão representados como a sequência das cidades percorridas, onde cada parte do cromossomo é um número inteiro indicando uma cidade.
Com isso, faremos o crossover da seguinte maneira:

In [6]:
def crossover(pai1, pai2):
    ponto_corte1 = rd.randint(1, num_cidades - 1)
    ponto_corte2 = rd.randint(ponto_corte1 + 1, num_cidades)
    filho = [-1] * num_cidades
    filho[ponto_corte1:ponto_corte2] = pai1[ponto_corte1:ponto_corte2]

    # Preenche o restante do filho com genes do pai2
    idx_pai2 = 0
    for i in range(num_cidades):
        if filho[i] == -1:
            while pai2[idx_pai2] in filho:
                idx_pai2 += 1
            filho[i] = pai2[idx_pai2]

    return filho

A última função que iremos criar é a de mutação, responsável por garantir uma variabilidade genética, trocando alguns genes de posição.

In [8]:
def mutacao(individuo, depot):
    idx1, idx2 = rd.sample(range(1, num_cidades), 2)  # Evitar o depot na mutação
    individuo[idx1], individuo[idx2] = individuo[idx2], individuo[idx1]

    # Garantir que o depot esteja sempre na primeira posição
    idx_depot = individuo.index(depot)
    individuo[0], individuo[idx_depot] = individuo[idx_depot], individuo[0]

    return individuo

Devemos ajustar os parâmetros para obtermos resultados, em média, melhores.

In [9]:
# Parâmetros para o A.G.
tamanho_populacao = 100
taxa_crossover = 0.8
taxa_mutacao = 0.1
geracoes = 100

Finalmente, aplicamos o Algoritmo Genético

In [10]:
num_cidades = len(data["distance_matrix"])

populacao = criar_populacao_inicial(tamanho_populacao, data["depot"])


for geracao in range(geracoes):
    nova_populacao = []

    for _ in range(tamanho_populacao // 2):
        pai1 = selecionar_pais(populacao, 5)
        pai2 = selecionar_pais(populacao, 5)

        filho1 = crossover(pai1, pai2)
        filho2 = crossover(pai2, pai1)

        filho1 = mutacao(filho1, data["depot"]) if rd.uniform(0, 1) < taxa_mutacao else filho1
        filho2 = mutacao(filho2, data["depot"]) if rd.uniform(0, 1) < taxa_mutacao else filho2


        nova_populacao.extend([filho1, filho2])

    populacao = nova_populacao

melhor_caminho = min(populacao, key=fitness)
melhor_distancia = fitness(melhor_caminho)

print("Melhor Caminho:", melhor_caminho)
print("Melhor Distância:", melhor_distancia)

# Nomes das cidades
nomes_cidades = [
    "New York", "Los Angeles", "Chicago", "Minneapolis", "Denver", "Dallas",
    "Seattle", "Boston", "San Francisco", "St. Louis", "Houston", "Phoenix", "Salt Lake City"
]

# Exibir o caminho em termos de cidades
caminho_cidades = [nomes_cidades[cidade] for cidade in melhor_caminho]

print("Melhor Caminho em Termos de Cidades:")
print(caminho_cidades)

Melhor Caminho: [0, 7, 3, 11, 1, 8, 6, 12, 4, 5, 10, 9, 2]
Melhor Distância: 7668
Melhor Caminho em Termos de Cidades:
['New York', 'Boston', 'Minneapolis', 'Phoenix', 'Los Angeles', 'San Francisco', 'Seattle', 'Salt Lake City', 'Denver', 'Dallas', 'Houston', 'St. Louis', 'Chicago']


# Observação
O problema do Caixeiro Viajante (TSP) é um problema NP-difícil, o que significa que não existe um algoritmo conhecido que resolva eficientemente todos os casos em tempo polinomial. Portanto, não é possível garantir sempre a menor distância possível para instâncias grandes do problema.

O que pode fazer é ajustar os parâmetros para encontrar, em média, o melhor caminho.